# Test treatment effects under placebo

The set up for the following tests is to test the regression estimator for the treatment effect in the case that the treatment effect is 0, but the treatment is related to the risk factor affecting the outcome.


## Example A.

Simplest possible example.  One binary risk factor $X$, one binary treatment $T$, one binary outcome $Y$.

$\begin{align}
P(X=1) = 0.25 \\
P(T=1 \mid X=1) = 0.95 \\
P(T=1 \mid X=0) = 0.05 \\
P(Y=1 \mid X=1) = 0.95 \\
P(Y=1 \mid X=0) = 0.05
\end{align}$

We generate N=200 patients, then estimate the treatment effect.  This is repeated n_iter = 1000 times for the bootstrap 95% confidence interval.

In [8]:
N = 200
p_has_risk_factor = 0.25
p_treat_given_risk_factor = 0.95
p_treat_given_no_risk_factor = 0.05
p_outcome_given_risk_factor = 0.95
p_outcome_given_no_risk_factor = 0.05
n_iter = 1000

### Example A. The regression estimator.

We fit $\mathrm{logit}(Y) \sim \beta X + \tau T$.  The estimated parameter $\hat{\tau}$ is the regression estimator for the treatment effect.

In [9]:
from treatment_effect_under_placebo_simulations import scenario_A, treatment_effect_estimator
method = 'regression'   
cis, _ = scenario_A(N,
                    n_iter,
                    p_has_risk_factor,
                    p_treat_given_risk_factor,
                    p_treat_given_no_risk_factor,
                    p_outcome_given_risk_factor,
                    p_outcome_given_no_risk_factor,
                    method)
print('Treatment Effect CI: ({:.2f},  {:.2f})'.format(cis[0],cis[1]))

Treatment Effect CI: (0.09,  0.29)


This scenario works as follows. 

In [10]:
# First, generate the risk factor, treatment, outcome variables.
import numpy as np
x = np.random.binomial(1,p_has_risk_factor,size=(N,))
treat = np.random.binomial(1,p_treat_given_risk_factor*x+p_treat_given_no_risk_factor*(1-x))         
y = np.random.binomial(1,p_outcome_given_risk_factor*x+p_outcome_given_no_risk_factor*(1-x))

# Summarize
X = np.vstack([x, treat]).T
import pandas as pd
df = pd.DataFrame(X, columns=['risk factor', 'treatment'])
print(df.groupby(['risk factor', 'treatment']).size())

# Second, predict the outcome using logistic regression.
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2')  
clf.fit(X,y)

# We use the G-computation estimator for the treatment effect
estimated_treatment_effect = treatment_effect_estimator(X,clf,method='gcomp')
print('Estimated treatment effect = {:.2f}'.format(estimated_treatment_effect))

risk factor  treatment
0            0            143
             1              5
1            0              4
             1             48
dtype: int64
Estimated treatment effect = 0.16


### Example A. Exact matching, followed by applying the G-computation estimator

In [11]:
# Sample matches between treatment and control groups.
from treatment_effect_under_placebo_simulations import match_exact
idx = match_exact(x, treat)
X = np.vstack([x, treat]).T[idx,:]  
y_ = y[idx]

# Summarize
df = pd.DataFrame(X, columns=['risk factor', 'treatment'])
print(df.groupby(['risk factor', 'treatment']).size())

# Then predict the outcome using logistic regression.
clf.fit(X,y_)

# Use G-computation estimator for the treatment effect.
estimated_treatment_effect = treatment_effect_estimator(X,clf,method='gcomp')
print('Estimated treatment effect = {:.2f}'.format(estimated_treatment_effect))

risk factor  treatment
0            0            5
             1            5
1            0            4
             1            4
dtype: int64
Estimated treatment effect = -0.04


### Example A. Distance matching, followed by applying the G-computation estimator

In [12]:
# Sample matches between treatment and control groups.
from treatment_effect_under_placebo_simulations import match_dist
idx = match_dist(x, treat, caliper=0.9) 
X = np.vstack([x, treat]).T[idx,:]  
y_ = y[idx]

# Summarize
df = pd.DataFrame(X, columns=['risk factor', 'treatment'])
print(df.groupby(['risk factor', 'treatment']).size())

# Then predict the outcome using logistic regression.
clf.fit(X,y_)

# Estimate treatment effect.
estimated_treatment_effect = treatment_effect_estimator(X,clf,method='gcomp')
print('Estimated treatment effect = {:.2f}'.format(estimated_treatment_effect))

risk factor  treatment
0            0             5
             1             5
1            0            48
             1            48
dtype: int64
Estimated treatment effect = -0.03


In [13]:
# idx

### Example A. Run all estimators for treatment effect CIs.

In [14]:
# Parameter Settings
N = 200
p_has_risk_factor = 0.25
p_treat_given_risk_factor = 0.95
p_treat_given_no_risk_factor = 0.05
p_outcome_given_risk_factor = 0.95
p_outcome_given_no_risk_factor = 0.05
n_iter = 1000

# Run all methods
methods = ['regression', 'matchexact', 'matchdist, greedy', 'matchdist, 1-1']
for method in methods:
    cis, num_valid = scenario_A(N,
                                n_iter,
                                p_has_risk_factor,
                                p_treat_given_risk_factor,
                                p_treat_given_no_risk_factor,
                                p_outcome_given_risk_factor,
                                p_outcome_given_no_risk_factor,
                                method)
    print('Method {}, Treatment Effect CI: ({:.4f},  {:.4f}); {} valid runs.'.format(method,cis[0],cis[1],num_valid))

Method regression, Treatment Effect CI: (0.0901,  0.2942); 1000 valid runs.
Method matchexact, Treatment Effect CI: (-0.1314,  0.0362); 951 valid runs.
Method matchdist, greedy, Treatment Effect CI: (-0.0971,  0.6510); 952 valid runs.
Method matchdist, 1-1, Treatment Effect CI: (-0.1283,  0.0416); 957 valid runs.
